# <center style="color: #FF5A60"> Unveiling the Shifting Landscape of Airbnb:<br>Understanding Supply, Demand, and Pricing Dynamics </center> <a class='tocSkip'>
    
This notebook is dedicated for the details of the data collection and cleaning process that serves as a supplementary to the main report of this project.
    
**! Note:** Rerunning this whole notebook, especially the cells under the `Code` section, is not advisable as this would (1) overwrite the current files generated, and (2) take almost 2 days to finish.

In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import pyspark.pandas as ps
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master('local[*]')
         .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
         .getOrCreate())
spark.sparkContext.setLogLevel('OFF')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/15 22:05:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## I. Background

The analysis looks at the availability and pricing of listings in Airbnb from 2019-2021 across 10 countries namely: France, Italy, Spain, United Kingdom, United States, Belgium, South Africa, Ireland, Switzerland, and Japan. This information is found in their database's **calendar dataset**.


**Challenges**

The data cleaning and preprocessing pipeline were motivated by these 5 challenges:

1. As detailed in [`Inside Airbnb`](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=150111846)'s official data dictionary, the calendar dataset was revised in 2019 to add new features such as `adjusted_price`, `minimum_nights`, and `maximum_nights`. This change introduced complexities when merging files across different countries, states, and years since the new features could be absent in one file while present in the other.
<br>

2. The calendar dataset does not have details on the location of the listing so deriving this information from another source is needed for the comparative analysis on different countries.
<br>

3. Some of the files are malformed having unescaped quotes or inconsistent use of delimiter as an escape character. This resulted in an error when loading data in the same year, country, or state in bulk since some would have a different data structure than others.
<br>

4. Some of the fields in a file need to be transformed into a different data type first before mathematical operations can be applied. For example, `price` is read as a string instead of a number since it has characters indicating currency such as dollar signs, commas, and periods. Moreover, other fields such as `minimum_nights` and `maximum_nights` are also read as a string due to missing value tags such as "None".
<br>

5. A total of around 21.8Gb data was used in this project and the data for a whole year has billions of rows. Due to this, data transformations take a long time to run and operations were done inefficiently. 


In [2]:
print("A sample set of the calendar dataset's 2020 files")
t1 = ps.read_csv(
    '/mnt/data/public/insideairbnb/data.insideairbnb.com/united-states/'\
    'ca/los-angeles/2020-12-15/data/calendar.csv.gz'
    )
t1.head()

A sample set of the calendar dataset's 2020 files


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,106052,2020-12-17,f,$66.00,$66.00,5,1125
1,109,2020-12-16,f,$115.00,$115.00,30,730
2,109,2020-12-17,f,$115.00,$115.00,30,730
3,109,2020-12-18,f,$115.00,$115.00,30,730
4,109,2020-12-19,f,$115.00,$115.00,30,730


In [3]:
print('Total file size of the data used:')
! du -sch --total \
/mnt/data/public/insideairbnb/data.insideairbnb.com/france/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/italy/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/spain/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/united-kingdom/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/united-states/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/belgium/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/south-africa/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/ireland/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/switzerland/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/japan/*/*/2019*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/france/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/italy/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/spain/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/united-kingdom/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/united-states/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/belgium/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/south-africa/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/ireland/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/switzerland/*/*/202[0-1]*/data/calendar.csv.gz \
/mnt/data/public/insideairbnb/data.insideairbnb.com/japan/*/*/202[0-1]*/data/calendar.csv.gz | tail -n 1

Total file size of the data used:
22G	total


**Solution**

To address these challenges, the following steps were done:

1. The data was collected per country and year instead of reading the whole in one go to ensure that the files accessed would have the same data structure.
<br>

2. The country where the listings are located was inferred from the name of the directory where the file being read is. For example, given the file path: `/mnt/data/public/insideairbnb/data.insideairbnb.com/united-states/ca/*/2020*/data/calendar.csv.gz`, the country was determined by separating the name by the backslash ('/') character then getting the 6th token.
<br>

3. Malformed files and rows in the dataset were skipped and dropped, respectively to ensure the information read was consistent and usable for analysis.
<br>

4. Extra characters were removed and tags for missing values were standardized to have Numpy's `np.nan` as their value for consistency, allowing the fields to be read in their expected data types.
<br>

5. The dataset was loaded in batches based on the country and year in the listing date. From CSV files, each batch was then converted to Parquet files to facilitate more efficient processing when doing transformations and analysis.

In [4]:
print("A sample set of the calendar dataset's 2020 files after cleaning " \
      "and preprocessing:")
t2 = ps.read_parquet(
    '/mnt/processed/private/msds2023/cpt1/airbnb_data/calendar/2020/'\
    'united-states/part-00000-6aab4765-7dfc-4d05-8b1e-c86ca17b85c9-c000.snappy.parquet/'
    )
t2.head()

A sample set of the calendar dataset's 2020 files after cleaning and preprocessing:


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,year,month,dayofweek,country
0,38585,2020-02-29,f,50.0,50.0,1.0,3.0,2020,February,Saturday,United States
1,6004216,2020-02-29,t,50.0,50.0,1.0,7.0,2020,February,Saturday,United States
2,6004216,2020-03-01,t,50.0,50.0,1.0,7.0,2020,March,Sunday,United States
3,6004216,2020-03-02,t,48.0,48.0,1.0,7.0,2020,March,Monday,United States
4,6004216,2020-03-03,t,48.0,48.0,1.0,7.0,2020,March,Tuesday,United States


## II. Code

The following code blocks are used to collect and clean the data needed for this project. The countries and year to be processed and the expected data types per column were first defined. If the country has not been processed yet, the code would proceed. Otherwise, skip to the next one until done.

Malformed files and rows were also skipped when reading the data in bulk. Correctly formatted files were then cleaned by removing extra characters that would hinder them from being processed as the expected data type (e.g., converting `str` to `float` or `int`).

In [ ]:
# define the countries to be processed
countries = ['belgium', 'ireland', 'japan', 'switzerland', 'france',
             'italy', 'spain', 'united-kingdom', 'united-states']

# define the data types of each column in the dataset
col_dtypes = {'listing_id': int, 'price': str, 'adjusted_price': str,
              'minimum_nights': str, 'maximum_nights': str}
col_dates = ['date']

def clean_data(df):
    """
    Return the dataframe with extra characters removed, standardized
    missing value tags, and additional information on the listing's
    country location. 
    """
    df2 = (df
             .assign(available=df.available.replace({'N/A': np.nan, 
                                                     'None': np.nan}),
                     minimum_nights=df.minimum_nights
                                        .replace({'N/A': np.nan, 
                                                  'None': np.nan})
                                        .astype(float),
                     maximum_nights=df.maximum_nights
                                        .replace({'N/A': np.nan, 
                                                  'None': np.nan})
                                        .astype(float),
                     price=df.price.replace({'N/A': np.nan,
                                             'None': np.nan,
                                             ',': ''})
                                   .str.strip('$').astype(float),
                     adjusted_price=df.adjusted_price
                                      .replace({'N/A': np.nan,
                                               'None': np.nan,
                                               ',': ''})
                                      .str.strip('$').astype(float),
                     year=df.date.dt.year.astype('int32'),
                     month=df.date.dt.month_name(),
                     dayofweek=df.date.dt.day_name()
                    )  
    )
    df2['country'] = f.split('/')[6].replace('-', ' ').title()
    
    return df2

In [ ]:
# added to remove printed warnings and save on memory
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# stores the country and year when an error is encountered
error = []
# stores countries that has already been processed
done = []
# defines the years to be processed
year = [2019, 2020 , 2021]

for idx, c in enumerate(countries):
    
    # checks if country has already been processed
    if c not in done:
        
        print(f'{idx}: {c}')
        for y in year:
            print(y)
            f = '/mnt/data/public/insideairbnb/data.insideairbnb.com/'\
                f'{c}/*/*/{y}*/data/'\
                'calendar.csv.gz'

            # try reading the files per country and year
            try:
                psdf = ps.read_csv(f, sep=",", multiLine=True, mode="DROPMALFORMED",
                                   escapechar='"', quotechar='"',
                                   dtypes=col_dtypes)

                # clean the data
                psdf2 = clean_data(psdf)

                # define path for saving
                ctry = f.split('/')[6] 
                path = f'/mnt/processed/private/msds2023/cpt1/airbnb_data/calendar/{y}/'
                out_fname = f'{ctry}'

                # save to parquet
                (psdf2.to_parquet(path+out_fname, mode='overwrite'))

            # skip the file if an error is encountered and log it
            except:
                print(f'Error: {c}--{y}')
                error.append([c, y])
                continue

        done.append(c)
    
    # skips the country if it has already been processed
    else:
        continue

-End of Data Preparation Notebook-